In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import wandb
import /Users/floseifert/Desktop/Subiculum/Stuff/subiculum/code/libraries/Neural_Lib_Flo as nlf 
import /Users/floseifert/Desktop/Subiculum/Stuff/subiculum/code/libraries/Training_Library as tl
import /Users/floseifert/Desktop/Subiculum/Stuff/subiculum/code/libraries/wandb_library as wl 

In [ ]:
# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Set the root directory
root_dir_sensorium = '/project/data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6/data'
images_path = '/project/subiculum/data/images_uint8.npy'
v1_responses_path = '/project/subiculum/data/V1_Data.mat'
sub_responses_path= '/project/subiculum/data/Post_Sub_Data.mat'

I have already run a sweep using wandb, so this won't be done again here.

In [ ]:
api = wandb.Api()
run = api.run("seifertflo/V1 Training 17-06-2024/2mr3hewa")
config=run.config
v1_model=nlb.configure_model(config, 13, device)
v1_train_loader, v1_val_loader, v1_test_loader = nlb.dataloader_from_mat(images_path, v1_responses_path, 75, 125, 64)
nlb.train_and_eval(model, config.get("epochs"), v1_train_loader, v1_test_loader, v1_val_loader, device)

We can now use the same configuration to get a rough idea of what training with the Sub Data would give:

In [ ]:
sub_train_loader, sub_val_loader, sub_test_loader =nlb.dataloader_from_mat(images_path, sub_responses_path, 75, 125, 64)
sub_model=nlb.configure_model(config, 37, device)
nlb.train_and_eval(sub_model, config.get("epochs"),sub_train_loader, sub_val_loader, sub_test_loader, device), 

As one gets a correlation of ~30% for the Sensorium data, it would be helpful to understand why this is not reached for the v1 data here. 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Load Images to np.array
images = np.load(images_path)

# Load responses and preprocess them
v1_responses, _, _ = load_mat_file(v1_responses_path)
v1_responses = np.transpose(v1_responses, (1, 2, 0))

# Look at mean data for neurons
v1_responses_mean = np.mean(v1_responses, axis=0, keepdims=True)

# Define the number of plots
num_plots = 13
neuron_indices = range(num_plots)

# Create a 3x5 grid of subplots
fig, axs = plt.subplots(3, 5, figsize=(15, 9))
axs = axs.flatten()  # Flatten the 2D array to 1D for easier iteration

# Create a custom legend patch
legend_patch = mpatches.Patch(color='gray', alpha=0.5, label='Image shown')

# Plot each neuron data
x = np.arange(0, 2000, 10)
for i, neuron_index in enumerate(neuron_indices):
    if i < len(neuron_indices):  # Ensure we don't go out of bounds
        neuron_mean = v1_responses_mean[:, :, neuron_index]
        axs[i].plot(x, neuron_mean[0, :])
        axs[i].axvspan(750, 1250, color='gray', alpha=0.5)  # Add transparent gray tile
        axs[i].set_title(f'Neuron {neuron_index + 1}')
        axs[i].set_xlabel('Time (ms)')
        axs[i].set_ylabel('Response')
        # Add the legend to the first plot only to avoid repetition
        if i == 0:
            axs[i].legend(handles=[legend_patch])

# Remove empty subplots (if any)
for j in range(len(neuron_indices), len(axs)):
    fig.delaxes(axs[j])

# Adjust layout
plt.tight_layout()
plt.show()

This shows the structure of the responses.
Now Ill compute oracle of the sensorium Data to compare the best neurons to the neurons here.

In [ ]:
sensorium_train_loader, sensorium_val_loader, sensorium_test_loader = nlb.dataloader_from_npy_pretraining(root_dir_sensorium, device)
sensorium_model = 

